
# 📈 Señales de Trading Local (4h) — Momentum Integral
Este notebook replica la lógica de tu app de Streamlit, pero para correr **localmente** en Jupyter:
- Descarga velas 4h desde Binance (endpoint público)
- Calcula EMAs, MACD, RSI y Stochastic RSI
- Calcula **Momentum Integral** y genera señales (`BUY`/`SELL`)
- Elimina señales consecutivas y **propaga** el último estado
- Grafica las **últimas 4 semanas** con velas + banderas de señales
- (Opcional) Mide desempeño simple con pares BUY→SELL

> **Requisitos:** `pandas`, `numpy`, `plotly`, `requests`, `pytz`  
> **Sugerencia:** corre cada celda en orden. 


In [1]:

import pandas as pd
import numpy as np
import requests
import time
from concurrent.futures import ThreadPoolExecutor, as_completed



In [2]:
# ==========================================================
# 🧠 Test Local — Momentum Integral + Market Flow + Alignment
# ==========================================================
import pandas as pd
import numpy as np
import requests
import plotly.graph_objects as go
import time

# ==========================================================
# 1️⃣ FETCH HISTÓRICO (velas 4H)
# ==========================================================
def get_binance_4h_data(symbol: str, limit: int = 300) -> pd.DataFrame:
    url = "https://api.binance.com/api/v3/klines"
    params = {"symbol": symbol, "interval": "4h", "limit": limit}
    r = requests.get(url, params=params)
    r.raise_for_status()
    data = r.json()

    cols = [
        "Open time","Open","High","Low","Close","Volume",
        "Close time","Quote asset volume","Number of trades",
        "Taker buy base asset volume","Taker buy quote asset volume","Ignore"
    ]
    df = pd.DataFrame(data, columns=cols)
    for c in ["Open","High","Low","Close","Volume"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    df["Open time"]  = pd.to_datetime(df["Open time"],  unit="ms", utc=True).dt.tz_convert("America/Costa_Rica")
    df["Close time"] = pd.to_datetime(df["Close time"], unit="ms", utc=True).dt.tz_convert("America/Costa_Rica")
    df = df.sort_values("Open time").reset_index(drop=True)
    return df

# ==========================================================
# 2️⃣ INDICADORES BASE (EMA + MOMENTUM INTEGRAL)
# ==========================================================
def calculate_indicators(df):
    df = df.copy()
    df["EMA20"] = df["Close"].ewm(span=20, adjust=False).mean()
    df["EMA50"] = df["Close"].ewm(span=50, adjust=False).mean()
    df["EMA200"] = df["Close"].ewm(span=200, adjust=False).mean()
    return df

def calcular_momentum_integral(df, window=6):
    df = df.copy()
    df["momentum"] = df["Close"].diff()
    df["integral_momentum"] = df["momentum"].rolling(window=window).sum()
    df["slope_integral"] = df["integral_momentum"].diff()
    std_slope = df["slope_integral"].rolling(window=window).std()

    df["Momentum Signal"] = np.where(
        (df["slope_integral"] < -std_slope) & (df["momentum"] < 0),
        "SELL",
        np.where(
            (df["slope_integral"] > std_slope) & (df["momentum"] > 0),
            "BUY",
            None
        )
    )
    return df

def limpiar_señales_consecutivas(df, columna="Momentum Signal"):
    df = df.copy()
    df["Signal Final"] = df[columna]
    for i in range(1, len(df)):
        if df.at[i, "Signal Final"] == df.at[i-1, "Signal Final"]:
            df.at[i, "Signal Final"] = None
    df["Signal Final"] = df["Signal Final"].ffill()
    return df

# ==========================================================
# 3️⃣ DETECCIÓN DE ABSORCIÓN (volumen alto, cuerpo chico)
# ==========================================================
def detectar_absorcion(df, vol_mult=1.5, body_ratio=0.25):
    df = df.copy()
    vol_prom = df["Volume"].rolling(10).mean()
    cuerpo = (df["Close"] - df["Open"]).abs()
    rango = df["High"] - df["Low"]
    df["Absorcion"] = (df["Volume"] > vol_prom * vol_mult) & (cuerpo < rango * body_ratio)
    return df

# ==========================================================
# 4️⃣ ORDER BOOK DEPTH (flow imbalance real)
# ==========================================================
def get_orderbook_imbalance(symbol: str, limit: int = 100) -> float:
    url = f"https://api.binance.com/api/v3/depth?symbol={symbol}&limit={limit}"
    try:
        r = requests.get(url, timeout=5)
        r.raise_for_status()
        data = r.json()
        bids = sum(float(b[1]) for b in data["bids"])
        asks = sum(float(a[1]) for a in data["asks"])
        if bids + asks == 0:
            return 0.0
        return round((bids - asks) / (bids + asks), 3)
    except Exception as e:
        print(f"⚠️ Error obteniendo depth: {e}")
        return 0.0

# ==========================================================
# 5️⃣ CONFIRMACIÓN FLOW / ABSORCIÓN
# ==========================================================
def confirmar_market_flow(df, symbol: str):
    df = df.copy()
    df["Flow Confirm"] = None
    df["Flow Strength"] = 0.0
    df["Order Imbalance"] = np.nan

    print(f"🔄 Consultando order book para {symbol}...")
    for i in range(len(df)):
        sig = df.at[i, "Signal Final"]
        absor = df.at[i, "Absorcion"]

        flow_imb = get_orderbook_imbalance(symbol)
        df.at[i, "Order Imbalance"] = flow_imb
        time.sleep(0.1)  # pausa leve para no saturar la API

        if sig == "BUY":
            if absor or flow_imb < 0.1:
                df.at[i, "Flow Confirm"] = "BUY ⚠️ Dudoso"
                df.at[i, "Flow Strength"] = 0.5
            else:
                df.at[i, "Flow Confirm"] = "BUY ✅ Confirmado"
                df.at[i, "Flow Strength"] = 1.0
        elif sig == "SELL":
            if absor or flow_imb > -0.1:
                df.at[i, "Flow Confirm"] = "SELL ⚠️ Dudoso"
                df.at[i, "Flow Strength"] = 0.5
            else:
                df.at[i, "Flow Confirm"] = "SELL ✅ Confirmado"
                df.at[i, "Flow Strength"] = 1.0
        else:
            df.at[i, "Flow Confirm"] = "⏸️ Sin señal"
            df.at[i, "Flow Strength"] = 0.0

    return df

# ==========================================================
# 6️⃣ MOMENTUM × FLOW ALIGNMENT SCORE
# ==========================================================
def calcular_alignment_score(df):
    df = df.copy()
    df["Momentum_Sign"] = np.sign(df["slope_integral"])
    df["Price_Change"] = df["Close"].diff()
    df["Price_Sign"] = np.sign(df["Price_Change"])
    if "Flow_MA" not in df.columns:
        df["Flow_MA"] = df["Order Imbalance"].rolling(window=10).mean()

    df["Alignment Score"] = (
        df["Momentum_Sign"] * df["Flow_MA"] * df["Price_Sign"]
    ).fillna(0)
    df["Alignment Score"] = df["Alignment Score"].clip(-1, 1)
    return df

# ==========================================================
# 7️⃣ VISUALIZACIÓN COMPLETA
# ==========================================================


def graficar_symbol_minimal(df, symbol, days=30):
    """
    Muestra solo las velas OHLC con puntos de compra/venta confirmados.
    Sin indicadores ni subgráficos adicionales.
    """
    fecha_limite = pd.Timestamp.now(tz=df["Open time"].dt.tz) - pd.Timedelta(days=days)
    df_filtrado = df[df["Open time"] >= fecha_limite].copy()

    fig = go.Figure()

    # --- Velas OHLC ---
    fig.add_trace(go.Candlestick(
        x=df_filtrado["Open time"],
        open=df_filtrado["Open"],
        high=df_filtrado["High"],
        low=df_filtrado["Low"],
        close=df_filtrado["Close"],
        name="Candlestick",
        increasing_line_color="#00FF00",  # verde brillante
        decreasing_line_color="#FF3333"   # rojo brillante
    ))

    # --- Señales (BUY / SELL) ---
    for _, row in df_filtrado.iterrows():
        signal = str(row.get("Flow Confirm", ""))
        if "BUY" in signal:
            color = "🟢" if "Confirmado" in signal else "🟡"
            y_pos = row["Low"]
            fig.add_trace(go.Scatter(
                x=[row["Open time"]],
                y=[y_pos],
                mode="text",
                text=[color],
                textposition="bottom center",
                showlegend=False
            ))
        elif "SELL" in signal:
            color = "🔴" if "Confirmado" in signal else "🟠"
            y_pos = row["High"]
            fig.add_trace(go.Scatter(
                x=[row["Open time"]],
                y=[y_pos],
                mode="text",
                text=[color],
                textposition="top center",
                showlegend=False
            ))

    # --- Layout limpio ---
    fig.update_layout(
        height=600,
        title=f"📊 {symbol} — Señales Confirmadas",
        template="plotly_dark",
        xaxis_rangeslider_visible=False,
        yaxis=dict(title="Precio")
    )

    fig.show()


# ==========================================================
# 8️⃣ PIPELINE LOCAL
# ==========================================================
def procesar_symbol_local(symbol="BTCUSDT", limit=200, window=6):
    df = get_binance_4h_data(symbol, limit)
    df = calculate_indicators(df)
    df = calcular_momentum_integral(df, window)
    df = limpiar_señales_consecutivas(df)
    df = detectar_absorcion(df)
    df = confirmar_market_flow(df, symbol)
    df = calcular_alignment_score(df)
    return df

# ==========================================================
# 9️⃣ MAIN
# ==========================================================
if __name__ == "__main__":
    symbol = "BTCUSDT"
    df = procesar_symbol_local(symbol)
    display(df[["Open time", "Signal Final", "Absorcion", "Order Imbalance", "Flow Confirm", "Alignment Score"]].tail(10))
    graficar_symbol_minimal(df, symbol)


🔄 Consultando order book para BTCUSDT...


,Open time,Signal Final,Absorcion,Order Imbalance,Flow Confirm,Alignment Score
190,2025-10-27 02:00:00-06:00,SELL,False,0.820,SELL ⚠️ Dudoso,0.7323
191,2025-10-27 06:00:00-06:00,SELL,False,0.799,SELL ⚠️ Dudoso,0.7544
192,2025-10-27 10:00:00-06:00,SELL,False,0.787,SELL ⚠️ Dudoso,-0.7811
193,2025-10-27 14:00:00-06:00,SELL,False,0.772,SELL ⚠️ Dudoso,0.8058
194,2025-10-27 18:00:00-06:00,SELL,False,0.761,SELL ⚠️ Dudoso,0.8275
195,2025-10-27 22:00:00-06:00,SELL,False,0.761,SELL ⚠️ Dudoso,-0.8158
196,2025-10-28 02:00:00-06:00,SELL,False,0.744,SELL ⚠️ Dudoso,0.8030
197,2025-10-28 06:00:00-06:00,SELL,False,0.771,SELL ⚠️ Dudoso,0.7929
198,2025-10-28 10:00:00-06:00,SELL,False,0.753,SELL ⚠️ Dudoso,0.7824
199,2025-10-28 14:00:00-06:00,SELL,False,0.729,SELL ⚠️ Dudoso,0.7697


In [3]:
# ==========================================================
# 🔍 Comparación de Estrategias Clásicas vs Nuestro Modelo
# ==========================================================

def calcular_macd(df, span_fast=12, span_slow=26, signal_span=9):
    ema_fast = df["Close"].ewm(span=span_fast, adjust=False).mean()
    ema_slow = df["Close"].ewm(span=span_slow, adjust=False).mean()
    macd = ema_fast - ema_slow
    signal = macd.ewm(span=signal_span, adjust=False).mean()
    df["MACD"] = macd
    df["MACD_Signal"] = signal
    df["MACD_Buy"]  = (macd > signal) & (macd.shift(1) <= signal.shift(1))
    df["MACD_Sell"] = (macd < signal) & (macd.shift(1) >= signal.shift(1))
    return df

def calcular_rsi(df, period=14):
    delta = df["Close"].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(period).mean()
    avg_loss = loss.rolling(period).mean()
    rs = avg_gain / avg_loss
    df["RSI"] = 100 - (100 / (1 + rs))
    df["RSI_Buy"]  = df["RSI"] < 30
    df["RSI_Sell"] = df["RSI"] > 70
    return df

def calcular_stochastic(df, k_period=14, d_period=3):
    low_min = df["Low"].rolling(k_period).min()
    high_max = df["High"].rolling(k_period).max()
    df["%K"] = 100 * (df["Close"] - low_min) / (high_max - low_min)
    df["%D"] = df["%K"].rolling(d_period).mean()
    df["Stoch_Buy"]  = (df["%K"] > df["%D"]) & (df["%K"].shift(1) <= df["%D"].shift(1)) & (df["%K"] < 20)
    df["Stoch_Sell"] = (df["%K"] < df["%D"]) & (df["%K"].shift(1) >= df["%D"].shift(1)) & (df["%K"] > 80)
    return df

def calcular_ema_crossover(df, short=20, long=50):
    df["EMA20"] = df["Close"].ewm(span=short, adjust=False).mean()
    df["EMA50"] = df["Close"].ewm(span=long, adjust=False).mean()
    df["EMA_Buy"]  = (df["EMA20"] > df["EMA50"]) & (df["EMA20"].shift(1) <= df["EMA50"].shift(1))
    df["EMA_Sell"] = (df["EMA20"] < df["EMA50"]) & (df["EMA20"].shift(1) >= df["EMA50"].shift(1))
    return df

# ==========================================================
# 📈 Evaluación de performance (Hit Rate, Profit Factor)
# ==========================================================
def evaluar_estrategia(df, col_buy, col_sell, precio_col="Close"):
    """
    Evalúa una estrategia dada. Puede recibir nombres de columnas o Series booleanas.
    """
    # Si nos pasan el nombre de columna, extraemos la Serie
    if isinstance(col_buy, str):
        buys = df[col_buy].fillna(False)
    else:
        buys = col_buy.fillna(False)

    if isinstance(col_sell, str):
        sells = df[col_sell].fillna(False)
    else:
        sells = col_sell.fillna(False)

    pares = []
    en_compra = False
    precio_entrada = 0.0

    for i, row in df.iterrows():
        if buys.iloc[i] and not en_compra:
            precio_entrada = row[precio_col]
            en_compra = True
        elif sells.iloc[i] and en_compra:
            ganancia = row[precio_col] - precio_entrada
            pares.append(ganancia)
            en_compra = False

    if not pares:
        return dict(hit_rate=0, n=0, gain=0, loss=0, profit_factor=0)

    pares = np.array(pares)
    ganancias = pares[pares > 0]
    perdidas = pares[pares <= 0]

    hit_rate = 100 * len(ganancias) / len(pares)
    gain_mean = ganancias.mean() if len(ganancias) > 0 else 0
    loss_mean = perdidas.mean() if len(perdidas) > 0 else 0
    pf = (ganancias.sum() / abs(perdidas.sum())) if perdidas.sum() != 0 else np.inf

    return dict(hit_rate=hit_rate, n=len(pares), gain=gain_mean, loss=loss_mean, profit_factor=pf)


# ==========================================================
# 🚀 Ejecutar comparación
# ==========================================================
df = calcular_macd(df)
df = calcular_rsi(df)
df = calcular_stochastic(df)
df = calcular_ema_crossover(df)

# Evaluamos cada estrategia
resultados = {
    "Momentum+Flow": evaluar_estrategia(df, col_buy="Flow Confirm", col_sell="Flow Confirm"),
    "MACD": evaluar_estrategia(df, col_buy="MACD_Buy", col_sell="MACD_Sell"),
    "RSI": evaluar_estrategia(df, col_buy="RSI_Buy", col_sell="RSI_Sell"),
    "Stochastic": evaluar_estrategia(df, col_buy="Stoch_Buy", col_sell="Stoch_Sell"),
    "EMA Crossover": evaluar_estrategia(df, col_buy="EMA_Buy", col_sell="EMA_Sell")
}

# Ajustar caso especial para tu modelo (Flow Confirm)
resultados["Momentum+Flow"] = evaluar_estrategia(
    df,
    col_buy=df["Flow Confirm"].str.contains("BUY", na=False),
    col_sell=df["Flow Confirm"].str.contains("SELL", na=False)
)

# ==========================================================
# 🧾 Mostrar resultados comparativos
# ==========================================================
resumen = pd.DataFrame(resultados).T.round(3)
print("\n📊 Comparación de Estrategias (últimos datos disponibles):")
display(resumen)



📊 Comparación de Estrategias (últimos datos disponibles):


,hit_rate,n,gain,loss,profit_factor
Momentum+Flow,45.455,11.0,4080.756,-1950.568,1.743
MACD,50.000,6.0,5872.437,-2062.017,2.848
RSI,0.000,1.0,0.000,-8174.800,0.000
Stochastic,50.000,2.0,5264.360,-10580.310,0.498
EMA Crossover,100.000,1.0,3372.260,0.000,inf


In [4]:
# ==========================================================
# 💰 BACKTEST con gestión fraccional de capital
# ==========================================================
import plotly.graph_objects as go

def backtest_fractional(df, buy_mask, sell_mask, precio_col="Close",
                        capital_inicial=710, sl=-0.02, tp=0.04, risk_fraction=0.1):
    """
    Simula un backtest con fracción de capital arriesgada por trade.
    El capital disponible se actualiza dinámicamente.
    """
    capital = capital_inicial
    capital_curve = [capital]
    posiciones = []
    en_trade = False
    precio_entrada = 0.0
    capital_en_trade = 0.0

    for i in range(1, len(df)):
        precio = df[precio_col].iloc[i]

        # abrir trade
        if buy_mask.iloc[i] and not en_trade:
            en_trade = True
            precio_entrada = precio
            capital_en_trade = capital * risk_fraction  # solo arriesgamos 10%
            continue

        if en_trade:
            cambio_pct = (precio - precio_entrada) / precio_entrada

            # TP o SL
            if cambio_pct >= tp or cambio_pct <= sl or sell_mask.iloc[i]:
                ganancia = capital_en_trade * cambio_pct
                capital += ganancia
                posiciones.append(cambio_pct)
                en_trade = False

        capital_curve.append(capital)

    # Cierre final si queda una operación abierta
    if en_trade:
        cambio_final = (df[precio_col].iloc[-1] - precio_entrada) / precio_entrada
        capital += capital_en_trade * cambio_final
        posiciones.append(cambio_final)

    # Métricas
    if not posiciones:
        return dict(final_capital=capital, n_trades=0, avg_gain_pct=0, hit_rate_pct=0), pd.Series(capital_curve)

    posiciones = np.array(posiciones)
    hits = posiciones > 0
    resultado = {
        "final_capital": round(capital, 2),
        "n_trades": len(posiciones),
        "avg_gain_pct": round(posiciones.mean() * 100, 2),
        "hit_rate_pct": round(100 * hits.mean(), 2)
    }
    return resultado, pd.Series(capital_curve)


# ==========================================================
# 🚀 Ejecutar backtests (Momentum+Flow vs MACD)
# ==========================================================
RISK = 0.1   # 10% del capital por trade
SL = -0.02
TP = 0.04

buy_flow  = df["Flow Confirm"].str.contains("BUY",  na=False)
sell_flow = df["Flow Confirm"].str.contains("SELL", na=False)
buy_macd  = df["MACD_Buy"].fillna(False)
sell_macd = df["MACD_Sell"].fillna(False)

res_flow, curve_flow = backtest_fractional(df, buy_flow, sell_flow, sl=SL, tp=TP, risk_fraction=RISK)
res_macd, curve_macd = backtest_fractional(df, buy_macd, sell_macd, sl=SL, tp=TP, risk_fraction=RISK)

# ==========================================================
# 📊 Resultados comparativos
# ==========================================================
resumen_frac = pd.DataFrame([res_flow, res_macd], index=["Momentum+Flow", "MACD"])
display(resumen_frac)

# ==========================================================
# 📈 Gráfico equity compuesto
# ==========================================================
fig = go.Figure()
fig.add_trace(go.Scatter(y=curve_flow, mode="lines", name="Momentum+Flow",
                         line=dict(color="#00cc96", width=3)))
fig.add_trace(go.Scatter(y=curve_macd, mode="lines", name="MACD",
                         line=dict(color="#ff6600", width=2, dash="dot")))

fig.update_layout(
    title=f"📈 Curva de Capital (10% por trade, SL={SL*100:.1f}%, TP={TP*100:.1f}%)",
    xaxis_title="Índice de Vela",
    yaxis_title="Capital (USD)",
    template="plotly_dark",
    height=500
)
fig.show()


,final_capital,n_trades,avg_gain_pct,hit_rate_pct
Momentum+Flow,715.4,13,0.59,46.15
MACD,716.5,6,1.53,50.00


In [35]:
# ==========================================================
# 📦 BLOQUE 1 — Descarga 5 símbolos + flow imbalance por vela
# ==========================================================
import pandas as pd
import numpy as np
import requests
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

# --- 1️⃣ Configuración general ---
symbols = ["BTCUSDT", "ETHUSDT", "ADAUSDT", "XRPUSDT", "BNBUSDT"]
limit = 1000          # ~6 meses (4 H)
depth_limit = 100     # profundidad del order book
snapshots_n = 10      # snapshots extra para medir flujo medio global
delay_s = 0.5         # pausa entre lecturas
BASE_URL = "https://api.binance.com"

# --- 2️⃣ Función para descargar velas históricas ---
def get_binance_4h_data(symbol: str, limit: int = 1000):
    url = f"{BASE_URL}/api/v3/klines"
    params = {"symbol": symbol, "interval": "4h", "limit": limit}
    r = requests.get(url, params=params)
    r.raise_for_status()
    data = r.json()
    cols = ["Open time","Open","High","Low","Close","Volume",
            "Close time","Quote asset volume","Number of trades",
            "Taker buy base asset volume","Taker buy quote asset volume","Ignore"]
    df = pd.DataFrame(data, columns=cols)
    for c in ["Open","High","Low","Close","Volume"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df["Open time"]  = pd.to_datetime(df["Open time"],  unit="ms", utc=True).dt.tz_convert("America/Costa_Rica")
    df["Close time"] = pd.to_datetime(df["Close time"], unit="ms", utc=True).dt.tz_convert("America/Costa_Rica")
    df = df.sort_values("Open time").reset_index(drop=True)
    return df

# --- 3️⃣ Función para obtener el order book actual ---
def get_orderbook_imbalance(symbol: str, depth: int = 100):
    url = f"{BASE_URL}/api/v3/depth"
    params = {"symbol": symbol, "limit": depth}
    try:
        r = requests.get(url, params=params, timeout=5)
        r.raise_for_status()
        data = r.json()
        bids = sum(float(b[1]) for b in data["bids"])
        asks = sum(float(a[1]) for a in data["asks"])
        if bids + asks == 0:
            return 0.0
        return round((bids - asks) / (bids + asks), 4)
    except Exception as e:
        print(f"⚠️ {symbol}: error leyendo orderbook -> {e}")
        return 0.0

# --- 4️⃣ Medir flujo medio global con varios snapshots ---
def snapshot_orderbook(symbol: str, n=10, delay=0.5):
    vals = []
    for _ in range(n):
        vals.append(get_orderbook_imbalance(symbol, depth_limit))
        time.sleep(delay)
    return [{"timestamp": time.time(), "value": v} for v in vals]

# --- 5️⃣ Procesar símbolo completo ---
def fetch_symbol_data(symbol):
    print(f"📥 {symbol} → descargando velas y orderbook por vela...")
    df = get_binance_4h_data(symbol, limit)

    # Flow imbalance por vela (simulado: una lectura por fila)
    flow_vals = []
    for _ in range(len(df)):
        flow_vals.append(get_orderbook_imbalance(symbol, depth_limit))
        time.sleep(0.1)  # evitar throttling

    df["Flow_Imbalance"] = flow_vals

    # snapshots globales para promedios
    snapshots = snapshot_orderbook(symbol, n=snapshots_n, delay=delay_s)
    flow_mean = np.mean([s["value"] for s in snapshots])
    flow_trend = snapshots[-1]["value"] - snapshots[0]["value"] if len(snapshots) > 1 else 0.0

    print(f"✅ {symbol}: {len(df)} velas | flow_mean={flow_mean:.3f} | flow_trend={flow_trend:.3f}")
    return symbol, {"df": df, "flow_snapshots": snapshots,
                    "flow_mean": flow_mean, "flow_trend_score": flow_trend}

# --- 6️⃣ Ejecutar descargas en paralelo ---
print("⚡ Descargando data y flow imbalance por vela...\n")
start = time.time()
data_full = {}

with ThreadPoolExecutor(max_workers=len(symbols)) as ex:
    futures = [ex.submit(fetch_symbol_data, s) for s in symbols]
    for f in as_completed(futures):
        sym, info = f.result()
        data_full[sym] = info

# --- 7️⃣ Guardar dataset completo ---
pd.to_pickle(data_full, "data_full_5coins_with_flow_per_candle.pkl")
print(f"\n💾 Guardado data_full_5coins_with_flow_per_candle.pkl ({len(data_full)} símbolos)")
print(f"⏱️ Tiempo total: {time.time() - start:.1f}s")


⚡ Descargando data y flow imbalance por vela...

📥 BTCUSDT → descargando velas y orderbook por vela...
📥 ETHUSDT → descargando velas y orderbook por vela...
📥 ADAUSDT → descargando velas y orderbook por vela...
📥 XRPUSDT → descargando velas y orderbook por vela...
📥 BNBUSDT → descargando velas y orderbook por vela...
✅ ADAUSDT: 1000 velas | flow_mean=0.154 | flow_trend=-0.057
✅ ETHUSDT: 1000 velas | flow_mean=-0.074 | flow_trend=0.452
✅ BNBUSDT: 1000 velas | flow_mean=-0.217 | flow_trend=0.243
✅ XRPUSDT: 1000 velas | flow_mean=0.115 | flow_trend=0.078
✅ BTCUSDT: 1000 velas | flow_mean=-0.009 | flow_trend=1.001

💾 Guardado data_full_5coins_with_flow_per_candle.pkl (5 símbolos)
⏱️ Tiempo total: 513.5s


In [26]:
# ==========================================================
# 💼 Backtest Symbol — RR 1:1.5 (versión simple y estable)
# ==========================================================
def backtest_symbol(df, buy_mask, sell_mask, price_col="Close",
                    capital_inicial=710, rr_tp=1.5, rr_sl=1.0, risk_fraction=1.0):
    """
    Simula trades BUY→SELL con TP:SL = 1.5:1.
    Usa risk_fraction para ponderar capital asignado al símbolo.
    """
    capital = capital_inicial * risk_fraction
    en_trade = False
    precio_entrada = 0.0
    capital_curve = [capital]

    for i in range(1, len(df)):
        precio = df[price_col].iloc[i]

        # Abrir posición BUY
        if buy_mask.iloc[i] and not en_trade:
            en_trade = True
            precio_entrada = precio
            continue

        # Cerrar trade si estamos dentro de uno
        if en_trade:
            cambio_pct = (precio - precio_entrada) / precio_entrada

            # Take-profit o stop-loss
            if cambio_pct >= rr_tp * 0.01 * 100 or cambio_pct <= -rr_sl * 0.01 * 100 or sell_mask.iloc[i]:
                capital += capital * cambio_pct
                en_trade = False

        capital_curve.append(capital)

    return capital, pd.Series(capital_curve)


In [36]:
# ==========================================================
# 🧠 BLOQUE 2 — Calcular flow_trend_score y comparar estrategias
# ==========================================================
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor

# 1️⃣ Cargar data generada por Bloque 1
data_full = pd.read_pickle("data_full_5coins_with_flow.pkl")

# 2️⃣ Calcular tendencia del flujo (flow_trend_score)
def compute_flow_trend(snapshots):
    vals = [s["value"] for s in snapshots if s.get("value") is not None]
    if len(vals) < 2:
        return 0.0
    return np.clip(vals[-1] - vals[0], -1, 1)

for sym, info in data_full.items():
    info["flow_trend_score"] = compute_flow_trend(info["flow_snapshots"])
    print(f"✅ {sym}: flow_mean={info['flow_mean']:.3f} | flow_trend={info['flow_trend_score']:.3f}")

# 3️⃣ Guardar dataset enriquecido
pd.to_pickle(data_full, "data_full_5coins_with_trend.pkl")
print("\n💾 Guardado como data_full_5coins_with_trend.pkl (con flow_trend_score)\n")

# 4️⃣ Funciones de indicadores y backtesting
def calcular_momentum_integral(df, window=6):
    df = df.copy()
    df["momentum"] = df["Close"].diff()
    df["integral_momentum"] = df["momentum"].rolling(window=window).sum()
    df["slope_integral"] = df["integral_momentum"].diff()
    std_slope = df["slope_integral"].rolling(window=window).std()
    df["Signal Final"] = np.select(
        [
            (df["slope_integral"] < -std_slope) & (df["momentum"] < 0),
            (df["slope_integral"] > std_slope) & (df["momentum"] > 0)
        ],
        ["SELL", "BUY"],
        default=None
    )
    return df

def confirmar_signal_con_flowtrend(df, flow_mean, flow_trend):
    """
    Confirma señales de Momentum según alineación con el flujo global y su tendencia.
    Más flexible: permite señales cuando el mercado está neutro o ligeramente alineado.
    """
    df = df.copy()
    df["Flow Confirm"] = "⏸️ Sin señal"

    for i in range(len(df)):
        sig = df.at[i, "Signal Final"]

        if sig == "BUY":
            # ✅ Confirmamos compra si el flujo no es fuertemente negativo
            if flow_mean > -0.05 and flow_trend > -0.05:
                df.at[i, "Flow Confirm"] = "BUY ✅ Confirmado"
            else:
                df.at[i, "Flow Confirm"] = "BUY ⚠️ Dudoso"

        elif sig == "SELL":
            # ✅ Confirmamos venta si el flujo no es fuertemente positivo
            if flow_mean < 0.05 and flow_trend < 0.05:
                df.at[i, "Flow Confirm"] = "SELL ✅ Confirmado"
            else:
                df.at[i, "Flow Confirm"] = "SELL ⚠️ Dudoso"

    return df


def add_macd(df, fast=12, slow=26, signal=9):
    df = df.copy()
    df["EMA_fast"] = df["Close"].ewm(span=fast, adjust=False).mean()
    df["EMA_slow"] = df["Close"].ewm(span=slow, adjust=False).mean()
    df["MACD"] = df["EMA_fast"] - df["EMA_slow"]
    df["Signal_Line"] = df["MACD"].ewm(span=signal, adjust=False).mean()
    df["MACD_Buy"] = (df["MACD"] > df["Signal_Line"]) & (df["MACD"].shift(1) <= df["Signal_Line"].shift(1))
    df["MACD_Sell"] = (df["MACD"] < df["Signal_Line"]) & (df["MACD"].shift(1) >= df["Signal_Line"].shift(1))
    return df

def backtest_symbol(df, col_buy, col_sell, capital_inicial=710, rr_tp=1.5, rr_sl=1.0):
    capital = capital_inicial
    position = None
    entry_price = 0.0
    for _, row in df.iterrows():
        price = row["Close"]
        if position is None and row[col_buy]:
            entry_price = price
            position = "long"
        elif position == "long" and row[col_sell]:
            tp = entry_price * (1 + rr_tp/100)
            sl = entry_price * (1 - rr_sl/100)
            if price >= tp:
                capital *= 1 + rr_tp/100
            elif price <= sl:
                capital *= 1 - rr_sl/100
            position = None
    return round(capital, 2)

# 5️⃣ Ejecutar backtest comparativo
pesos = {"BTCUSDT":0.35, "ETHUSDT":0.25, "ADAUSDT":0.10, "XRPUSDT":0.20, "BNBUSDT":0.10}

def procesar_symbol(symbol, info):
    df = info["df"]
    flow_mean = info["flow_mean"]
    flow_trend = info["flow_trend_score"]

    df_m = calcular_momentum_integral(df)
    df_m["Buy"] = df_m["Signal Final"] == "BUY"
    df_m["Sell"] = df_m["Signal Final"] == "SELL"
    cap_momentum = backtest_symbol(df_m, "Buy", "Sell")

    df_f = confirmar_signal_con_flowtrend(df_m, flow_mean, flow_trend)
    df_f["Buy"] = df_f["Flow Confirm"].str.contains("BUY ✅", na=False)
    df_f["Sell"] = df_f["Flow Confirm"].str.contains("SELL ✅", na=False)
    cap_flow = backtest_symbol(df_f, "Buy", "Sell")

    df_macd = add_macd(df)
    cap_macd = backtest_symbol(df_macd, "MACD_Buy", "MACD_Sell")

    return symbol, cap_momentum, cap_flow, cap_macd

print("⚙️ Ejecutando backtest en paralelo...\n")
start = pd.Timestamp.now()
results = {}

with ThreadPoolExecutor(max_workers=len(pesos)) as ex:
    for sym, cap_m, cap_f, cap_macd in ex.map(lambda s: procesar_symbol(s, data_full[s]), pesos.keys()):
        results[sym] = {"Momentum Integral": cap_m, "Momentum+Flow": cap_f, "MACD": cap_macd, "Peso": pesos[sym]}

df_res = pd.DataFrame(results).T
for col in ["Momentum Integral", "Momentum+Flow", "MACD"]:
    df_res[col] = df_res[col].astype(float)

df_res.loc["Portfolio Total"] = {
    "Momentum Integral": np.sum(df_res["Momentum Integral"] * df_res["Peso"]),
    "Momentum+Flow": np.sum(df_res["Momentum+Flow"] * df_res["Peso"]),
    "MACD": np.sum(df_res["MACD"] * df_res["Peso"]),
    "Peso": 1.0
}

print("\n📊 Resultados del Backtest (con Flow Trend Filter):\n")
display(df_res.round(2))
print(f"⏱️ Tiempo total: {(pd.Timestamp.now() - start).total_seconds():.2f} s")


✅ BTCUSDT: flow_mean=-0.211 | flow_trend=-0.144
✅ ETHUSDT: flow_mean=-0.196 | flow_trend=0.066
✅ ADAUSDT: flow_mean=0.098 | flow_trend=0.000
✅ XRPUSDT: flow_mean=0.107 | flow_trend=-0.071
✅ BNBUSDT: flow_mean=0.196 | flow_trend=-0.033

💾 Guardado como data_full_5coins_with_trend.pkl (con flow_trend_score)

⚙️ Ejecutando backtest en paralelo...


📊 Resultados del Backtest (con Flow Trend Filter):



,Momentum Integral,Momentum+Flow,MACD,Peso
BTCUSDT,708.41,710.0,654.17,0.35
ETHUSDT,743.53,710.0,729.82,0.25
ADAUSDT,639.72,710.0,666.95,0.10
XRPUSDT,751.32,710.0,650.77,0.20
BNBUSDT,790.33,710.0,814.83,0.10
Portfolio Total,727.10,710.0,689.75,1.00


⏱️ Tiempo total: 1.23 s


In [38]:
# ==========================================================
# 🧠 BLOQUE DE OPTIMIZACIÓN TOTAL (Multi-núcleo completo)
# ==========================================================
import pandas as pd
import numpy as np
import itertools
import time
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

# --- Configuración ---
data_full = pd.read_pickle("data_full_5coins_with_flow_per_candle.pkl")
pesos = {"BTCUSDT":0.35, "ETHUSDT":0.25, "ADAUSDT":0.10, "XRPUSDT":0.20, "BNBUSDT":0.10}
N_CORES = os.cpu_count()
print(f"🧮 Usando {N_CORES} núcleos disponibles.\n")

# ----------------------------------------------------------
# 1️⃣ Funciones auxiliares
# ----------------------------------------------------------
def calcular_momentum_integral(df, window):
    df = df.copy()
    df["momentum"] = df["Close"].diff()
    df["integral_momentum"] = df["momentum"].rolling(window=window).sum()
    df["slope_integral"] = df["integral_momentum"].diff()
    std_slope = df["slope_integral"].rolling(window=window).std()
    df["Signal Final"] = np.select(
        [
            (df["slope_integral"] < -std_slope) & (df["momentum"] < 0),
            (df["slope_integral"] > std_slope) & (df["momentum"] > 0)
        ],
        ["SELL", "BUY"],
        default=None
    )
    return df

def confirmar_signal_flow_per_candle(df, flow_threshold=0.05, alignment_weight=0.5):
    df = df.copy()
    df["Flow Confirm"] = "⏸️ Sin señal"
    for i in range(len(df)):
        sig = df.at[i, "Signal Final"]
        flow = df.at[i, "Flow_Imbalance"]
        if sig == "BUY" and flow > -flow_threshold * alignment_weight:
            df.at[i, "Flow Confirm"] = "BUY ✅ Confirmado"
        elif sig == "SELL" and flow < flow_threshold * (-alignment_weight):
            df.at[i, "Flow Confirm"] = "SELL ✅ Confirmado"
    return df

def backtest_symbol(df, col_buy, col_sell, rr_tp, rr_sl, capital_inicial=710):
    capital = capital_inicial
    position = None
    entry = 0.0
    for _, row in df.iterrows():
        price = row["Close"]
        if position is None and row[col_buy]:
            entry = price
            position = "long"
        elif position == "long" and row[col_sell]:
            tp = entry * (1 + rr_tp/100)
            sl = entry * (1 - rr_sl/100)
            if price >= tp:
                capital *= 1 + rr_tp/100
            elif price <= sl:
                capital *= 1 - rr_sl/100
            position = None
    return capital

# ----------------------------------------------------------
# 2️⃣ Evaluar parámetros en paralelo por símbolo
# ----------------------------------------------------------
def evaluar_parametros(window, rr_tp, rr_sl, flow_threshold, alignment_weight):
    resultados = {}

    # Ejecutamos los 5 símbolos en paralelo
    with ThreadPoolExecutor(max_workers=len(pesos)) as ex:
        futures = {}
        for sym, info in data_full.items():
            futures[ex.submit(evaluar_simbolo, sym, info, window, rr_tp, rr_sl, flow_threshold, alignment_weight)] = sym
        for f in as_completed(futures):
            sym, cap = f.result()
            resultados[sym] = cap

    # Capital ponderado del portafolio
    capital_port = np.sum([resultados[s] * pesos[s] for s in pesos])
    return {
        "window": window,
        "rr_tp": rr_tp,
        "rr_sl": rr_sl,
        "flow_threshold": flow_threshold,
        "alignment_weight": alignment_weight,
        "capital": round(capital_port, 2)
    }

def evaluar_simbolo(symbol, info, window, rr_tp, rr_sl, flow_threshold, alignment_weight):
    df = info["df"]
    df_m = calcular_momentum_integral(df, window)
    df_m = confirmar_signal_flow_per_candle(df_m, flow_threshold, alignment_weight)
    df_m["Buy"]  = df_m["Flow Confirm"].str.contains("BUY ✅", na=False)
    df_m["Sell"] = df_m["Flow Confirm"].str.contains("SELL ✅", na=False)
    cap = backtest_symbol(df_m, "Buy", "Sell", rr_tp, rr_sl)
    return symbol, cap

# ----------------------------------------------------------
# 3️⃣ Grid Search Total en paralelo
# ----------------------------------------------------------
param_grid = {
    "window": [4, 6, 8, 10, 12],
    "rr_tp": [1.0, 1.5, 2.0, 2.5],
    "rr_sl": [0.5, 1.0, 1.5],
    "flow_threshold": [0.0, 0.05, 0.1, 0.15],
    "alignment_weight": [0.3, 0.5, 0.7, 1.0]
}

param_combos = list(itertools.product(
    param_grid["window"],
    param_grid["rr_tp"],
    param_grid["rr_sl"],
    param_grid["flow_threshold"],
    param_grid["alignment_weight"]
))

print(f"🚀 Ejecutando {len(param_combos)} combinaciones con {N_CORES} núcleos...\n")
start = time.time()
results = []

with ThreadPoolExecutor(max_workers=N_CORES) as ex:
    futures = [ex.submit(evaluar_parametros, *p) for p in param_combos]
    for i, f in enumerate(as_completed(futures), 1):
        res = f.result()
        results.append(res)
        if i % 10 == 0:
            print(f"✅ {i}/{len(param_combos)} completadas")
        if i % 50 == 0:  # Guardar checkpoint
            pd.DataFrame(results).to_csv("opt_checkpoint.csv", index=False)

df_opt = pd.DataFrame(results).sort_values("capital", ascending=False)
df_opt.to_csv("opt_results_full.csv", index=False)

print(f"\n🏁 Optimización completada en {time.time() - start:.1f}s")
print("🏆 Mejores configuraciones encontradas:")
display(df_opt.head(10))


🧮 Usando 56 núcleos disponibles.

🚀 Ejecutando 960 combinaciones con 56 núcleos...

✅ 10/960 completadas
✅ 20/960 completadas
✅ 30/960 completadas
✅ 40/960 completadas
✅ 50/960 completadas
✅ 60/960 completadas
✅ 70/960 completadas
✅ 80/960 completadas
✅ 90/960 completadas
✅ 100/960 completadas
✅ 110/960 completadas
✅ 120/960 completadas
✅ 130/960 completadas
✅ 140/960 completadas
✅ 150/960 completadas
✅ 160/960 completadas
✅ 170/960 completadas
✅ 180/960 completadas
✅ 190/960 completadas
✅ 200/960 completadas
✅ 210/960 completadas
✅ 220/960 completadas
✅ 230/960 completadas
✅ 240/960 completadas
✅ 250/960 completadas
✅ 260/960 completadas
✅ 270/960 completadas
✅ 280/960 completadas
✅ 290/960 completadas
✅ 300/960 completadas
✅ 310/960 completadas
✅ 320/960 completadas
✅ 330/960 completadas
✅ 340/960 completadas
✅ 350/960 completadas
✅ 360/960 completadas
✅ 370/960 completadas
✅ 380/960 completadas
✅ 390/960 completadas
✅ 400/960 completadas
✅ 410/960 completadas
✅ 420/960 completadas
✅

,window,rr_tp,rr_sl,flow_threshold,alignment_weight,capital
679,8,2.5,0.5,0.15,1.0,774.65
12,10,2.5,0.5,0.00,0.7,765.88
211,10,2.5,0.5,0.00,0.3,765.88
853,10,2.5,0.5,0.00,0.5,765.88
745,10,2.5,0.5,0.00,1.0,765.88
281,6,2.5,0.5,0.15,1.0,764.53
83,8,2.5,0.5,0.00,0.5,762.32
746,8,2.5,0.5,0.00,0.7,762.32
139,8,2.5,0.5,0.00,0.3,762.32
757,8,2.5,0.5,0.00,1.0,762.32


In [39]:
# ==========================================================
# 📈 Visualización de señales optimizadas (Momentum vs Flow)
# ==========================================================
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# -----------------------------
# Cargar datos guardados
# -----------------------------
data_full = pd.read_pickle("data_full_5coins_with_flow.pkl")
symbol = "BTCUSDT"   # puedes cambiar a ETHUSDT, ADAUSDT, etc.
df = data_full[symbol]["df"]
flow_mean = data_full[symbol]["flow_mean"]

# -----------------------------
# Funciones base
# -----------------------------
def calcular_momentum_integral(df, window=8):
    df = df.copy()
    df["momentum"] = df["Close"].diff()
    df["integral_momentum"] = df["momentum"].rolling(window=window).sum()
    df["slope_integral"] = df["integral_momentum"].diff()
    std_slope = df["slope_integral"].rolling(window=window).std()

    df["Signal Final"] = np.select(
        [
            (df["slope_integral"] < -std_slope) & (df["momentum"] < 0),
            (df["slope_integral"] > std_slope) & (df["momentum"] > 0)
        ],
        ["SELL", "BUY"],
        default=None
    )
    return df

def confirmar_signal_con_flowtrend(df, flow_mean, flow_threshold=0.15, alignment_weight=1.0):
    df = df.copy()
    df["Flow Confirm"] = "⏸️ Sin señal"

    for i in range(len(df)):
        sig = df.at[i, "Signal Final"]

        if sig == "BUY":
            # Confirmamos solo si flujo promedio es claramente positivo
            if flow_mean > flow_threshold * alignment_weight:
                df.at[i, "Flow Confirm"] = "BUY ✅ Confirmado"
            else:
                df.at[i, "Flow Confirm"] = "BUY ⚠️ Dudoso"

        elif sig == "SELL":
            # Confirmamos solo si flujo promedio es claramente negativo
            if flow_mean < -flow_threshold * alignment_weight:
                df.at[i, "Flow Confirm"] = "SELL ✅ Confirmado"
            else:
                df.at[i, "Flow Confirm"] = "SELL ⚠️ Dudoso"

    return df

# -----------------------------
# Aplicar modelos
# -----------------------------
df = calcular_momentum_integral(df, window=8)
df_flow = confirmar_signal_con_flowtrend(df, flow_mean, flow_threshold=0.15, alignment_weight=1.0)

# -----------------------------
# Filtrar rango de tiempo (últimos 30 días)
# -----------------------------
fecha_limite = pd.Timestamp.now(tz=df["Open time"].dt.tz) - pd.Timedelta(days=30)
df_vis = df[df["Open time"] >= fecha_limite].copy()
df_flow_vis = df_flow[df_flow["Open time"] >= fecha_limite].copy()

# -----------------------------
# Gráfico interactivo
# -----------------------------
fig = go.Figure()

# --- Velas OHLC ---
fig.add_trace(go.Candlestick(
    x=df_vis["Open time"],
    open=df_vis["Open"],
    high=df_vis["High"],
    low=df_vis["Low"],
    close=df_vis["Close"],
    name="Velas",
    increasing_line_color="#00FF00",
    decreasing_line_color="#FF3333"
))

# --- Señales base ---
buy_base = df_vis[df_vis["Signal Final"] == "BUY"]
sell_base = df_vis[df_vis["Signal Final"] == "SELL"]

fig.add_trace(go.Scatter(
    x=buy_base["Open time"], y=buy_base["Low"],
    mode="markers", name="BUY (Momentum Base)",
    marker=dict(symbol="triangle-up", size=10, color="lime")
))

fig.add_trace(go.Scatter(
    x=sell_base["Open time"], y=sell_base["High"],
    mode="markers", name="SELL (Momentum Base)",
    marker=dict(symbol="triangle-down", size=10, color="red")
))

# --- Señales Flow Confirmadas ---
buy_flow = df_flow_vis[df_flow_vis["Flow Confirm"].str.contains("BUY ✅", na=False)]
sell_flow = df_flow_vis[df_flow_vis["Flow Confirm"].str.contains("SELL ✅", na=False)]

fig.add_trace(go.Scatter(
    x=buy_flow["Open time"], y=buy_flow["Low"],
    mode="markers", name="BUY (Flow Confirmed)",
    marker=dict(symbol="star-triangle-up", size=12, color="cyan")
))

fig.add_trace(go.Scatter(
    x=sell_flow["Open time"], y=sell_flow["High"],
    mode="markers", name="SELL (Flow Confirmed)",
    marker=dict(symbol="star-triangle-down", size=12, color="orange")
))

# --- Layout final ---
fig.update_layout(
    title=f"📊 {symbol} — Señales Momentum Integral vs Momentum+Flow (Optimizado)",
    xaxis_title="Fecha",
    yaxis_title="Precio (USDT)",
    template="plotly_dark",
    height=700,
    xaxis_rangeslider_visible=False,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig.show()



In [ ]:
# ==========================================================
# 🧠 Limpieza de señales consecutivas y visualización
# ==========================================================
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# --- Función para limpiar señales consecutivas ---
def limpiar_señales_consecutivas(df, columna="Signal Final"):
    df = df.copy()
    df["Signal Limpia"] = df[columna]
    for i in range(1, len(df)):
        if df.at[i, "Signal Limpia"] == df.at[i-1, "Signal Limpia"]:
            df.at[i, "Signal Limpia"] = None
    return df

# --- Función Momentum Integral ---
def calcular_momentum_integral(df, window=8):
    df = df.copy()
    df["momentum"] = df["Close"].diff()
    df["integral_momentum"] = df["momentum"].rolling(window=window).sum()
    df["slope_integral"] = df["integral_momentum"].diff()
    std_slope = df["slope_integral"].rolling(window=window).std()

    df["Signal Final"] = np.select(
        [
            (df["slope_integral"] < -std_slope) & (df["momentum"] < 0),
            (df["slope_integral"] > std_slope) & (df["momentum"] > 0)
        ],
        ["SELL", "BUY"],
        default=None
    )
    return df

# --- Función Flow Confirm ---
def confirmar_signal_con_flowtrend(df, flow_mean, flow_threshold=0.15, alignment_weight=1.0):
    df = df.copy()
    df["Flow Confirm"] = "⏸️ Sin señal"
    for i in range(len(df)):
        sig = df.at[i, "Signal Limpia"]
        if sig == "BUY":
            if flow_mean > flow_threshold * alignment_weight:
                df.at[i, "Flow Confirm"] = "BUY ✅ Confirmado"
            else:
                df.at[i, "Flow Confirm"] = "BUY ⚠️ Dudoso"
        elif sig == "SELL":
            if flow_mean < -flow_threshold * alignment_weight:
                df.at[i, "Flow Confirm"] = "SELL ✅ Confirmado"
            else:
                df.at[i, "Flow Confirm"] = "SELL ⚠️ Dudoso"
    return df

# --- Cargar datos ---
data_full = pd.read_pickle("data_full_5coins_with_flow.pkl")
symbol = "BTCUSDT"
df = data_full[symbol]["df"]
flow_mean = data_full[symbol]["flow_mean"]

# --- Aplicar pipeline completo ---
df = calcular_momentum_integral(df, window=8)
df = limpiar_señales_consecutivas(df, columna="Signal Final")

df_flow = confirmar_signal_con_flowtrend(df, flow_mean, flow_threshold=0.15, alignment_weight=1.0)
df_flow = limpiar_señales_consecutivas(df_flow, columna="Flow Confirm")

# --- Filtrar rango de visualización ---
fecha_limite = pd.Timestamp.now(tz=df["Open time"].dt.tz) - pd.Timedelta(days=30)
df_vis = df[df["Open time"] >= fecha_limite].copy()
df_flow_vis = df_flow[df_flow["Open time"] >= fecha_limite].copy()

# ==========================================================
# 📈 Gráfico final
# ==========================================================
fig = go.Figure()

# Velas
fig.add_trace(go.Candlestick(
    x=df_vis["Open time"],
    open=df_vis["Open"],
    high=df_vis["High"],
    low=df_vis["Low"],
    close=df_vis["Close"],
    name="Velas",
    increasing_line_color="#00FF00",
    decreasing_line_color="#FF3333"
))

# Momentum Base
buy_base = df_vis[df_vis["Signal Limpia"] == "BUY"]
sell_base = df_vis[df_vis["Signal Limpia"] == "SELL"]

fig.add_trace(go.Scatter(
    x=buy_base["Open time"], y=buy_base["Low"],
    mode="markers", name="BUY (Momentum Base)",
    marker=dict(symbol="triangle-up", size=10, color="lime")
))
fig.add_trace(go.Scatter(
    x=sell_base["Open time"], y=sell_base["High"],
    mode="markers", name="SELL (Momentum Base)",
    marker=dict(symbol="triangle-down", size=10, color="red")
))

# Flow Confirmed
buy_flow = df_flow_vis[df_flow_vis["Flow Confirm"].str.contains("BUY ✅", na=False)]
sell_flow = df_flow_vis[df_flow_vis["Flow Confirm"].str.contains("SELL ✅", na=False)]

fig.add_trace(go.Scatter(
    x=buy_flow["Open time"], y=buy_flow["Low"],
    mode="markers", name="BUY (Flow Confirmed)",
    marker=dict(symbol="star-triangle-up", size=12, color="cyan")
))
fig.add_trace(go.Scatter(
    x=sell_flow["Open time"], y=sell_flow["High"],
    mode="markers", name="SELL (Flow Confirmed)",
    marker=dict(symbol="star-triangle-down", size=12, color="orange")
))

fig.update_layout(
    title=f"📊 {symbol} — Señales Momentum Integral vs Flow (limpieza aplicada)",
    template="plotly_dark",
    height=700,
    xaxis_rangeslider_visible=False,
    yaxis_title="Precio (USDT)",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig.show()


In [41]:
# ==========================================================
# 🧠 Limpieza de señales persistentes + visualización
# ==========================================================
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# --- Función para limpiar señales persistentes ---
def limpiar_señales_persistentes(df, columna="Signal Final"):
    """
    Elimina señales repetidas del mismo tipo (BUY/SELL),
    incluso si hay velas intermedias sin señal.
    Solo emite una nueva señal cuando cambia de dirección.
    """
    df = df.copy()
    df["Signal Limpia"] = None
    last_signal = None

    for i in range(len(df)):
        current = df.at[i, columna]
        if current in ["BUY", "SELL"]:
            if current != last_signal:
                df.at[i, "Signal Limpia"] = current
                last_signal = current
            else:
                df.at[i, "Signal Limpia"] = None
        else:
            df.at[i, "Signal Limpia"] = None
    return df

# --- Función Momentum Integral ---
def calcular_momentum_integral(df, window=8):
    df = df.copy()
    df["momentum"] = df["Close"].diff()
    df["integral_momentum"] = df["momentum"].rolling(window=window).sum()
    df["slope_integral"] = df["integral_momentum"].diff()
    std_slope = df["slope_integral"].rolling(window=window).std()

    df["Signal Final"] = np.select(
        [
            (df["slope_integral"] < -std_slope) & (df["momentum"] < 0),
            (df["slope_integral"] > std_slope) & (df["momentum"] > 0)
        ],
        ["SELL", "BUY"],
        default=None
    )
    return df

# --- Función Flow Confirm ---
def confirmar_signal_con_flowtrend(df, flow_mean, flow_threshold=0.15, alignment_weight=1.0):
    df = df.copy()
    df["Flow Confirm"] = "⏸️ Sin señal"
    for i in range(len(df)):
        sig = df.at[i, "Signal Limpia"]
        if sig == "BUY":
            if flow_mean > flow_threshold * alignment_weight:
                df.at[i, "Flow Confirm"] = "BUY ✅ Confirmado"
            else:
                df.at[i, "Flow Confirm"] = "BUY ⚠️ Dudoso"
        elif sig == "SELL":
            if flow_mean < -flow_threshold * alignment_weight:
                df.at[i, "Flow Confirm"] = "SELL ✅ Confirmado"
            else:
                df.at[i, "Flow Confirm"] = "SELL ⚠️ Dudoso"
    return df

# --- Cargar datos ---
data_full = pd.read_pickle("data_full_5coins_with_flow.pkl")
symbol = "BTCUSDT"
df = data_full[symbol]["df"]
flow_mean = data_full[symbol]["flow_mean"]

# --- Aplicar pipeline completo ---
df = calcular_momentum_integral(df, window=8)
df = limpiar_señales_persistentes(df, columna="Signal Final")

df_flow = confirmar_signal_con_flowtrend(df, flow_mean, flow_threshold=0.15, alignment_weight=1.0)
df_flow = limpiar_señales_persistentes(df_flow, columna="Flow Confirm")

# --- Filtrar rango de visualización ---
fecha_limite = pd.Timestamp.now(tz=df["Open time"].dt.tz) - pd.Timedelta(days=30)
df_vis = df[df["Open time"] >= fecha_limite].copy()
df_flow_vis = df_flow[df_flow["Open time"] >= fecha_limite].copy()

# ==========================================================
# 📈 Gráfico final
# ==========================================================
fig = go.Figure()

# Velas
fig.add_trace(go.Candlestick(
    x=df_vis["Open time"],
    open=df_vis["Open"],
    high=df_vis["High"],
    low=df_vis["Low"],
    close=df_vis["Close"],
    name="Velas",
    increasing_line_color="#00FF00",
    decreasing_line_color="#FF3333"
))

# Momentum Base
buy_base = df_vis[df_vis["Signal Limpia"] == "BUY"]
sell_base = df_vis[df_vis["Signal Limpia"] == "SELL"]

fig.add_trace(go.Scatter(
    x=buy_base["Open time"], y=buy_base["Low"],
    mode="markers", name="BUY (Momentum Base)",
    marker=dict(symbol="triangle-up", size=10, color="lime")
))
fig.add_trace(go.Scatter(
    x=sell_base["Open time"], y=sell_base["High"],
    mode="markers", name="SELL (Momentum Base)",
    marker=dict(symbol="triangle-down", size=10, color="red")
))

# Flow Confirmed
buy_flow = df_flow_vis[df_flow_vis["Flow Confirm"].str.contains("BUY ✅", na=False)]
sell_flow = df_flow_vis[df_flow_vis["Flow Confirm"].str.contains("SELL ✅", na=False)]

fig.add_trace(go.Scatter(
    x=buy_flow["Open time"], y=buy_flow["Low"],
    mode="markers", name="BUY (Flow Confirmed)",
    marker=dict(symbol="star-triangle-up", size=12, color="cyan")
))
fig.add_trace(go.Scatter(
    x=sell_flow["Open time"], y=sell_flow["High"],
    mode="markers", name="SELL (Flow Confirmed)",
    marker=dict(symbol="star-triangle-down", size=12, color="orange")
))

fig.update_layout(
    title=f"📊 {symbol} — Señales Momentum Integral vs Flow (limpieza persistente aplicada)",
    template="plotly_dark",
    height=700,
    xaxis_rangeslider_visible=False,
    yaxis_title="Precio (USDT)",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig.show()


In [44]:
# ==========================================================
# 🧠 Limpieza persistente + Exportación a Excel
# ==========================================================
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# --- Función de limpieza persistente ---
def limpiar_señales_persistentes(df, columna="Signal Final"):
    """
    Elimina señales repetidas del mismo tipo (BUY/SELL),
    incluso si hay velas intermedias sin señal.
    Solo emite una nueva señal cuando cambia de dirección.
    """
    df = df.copy()
    df["Signal Limpia"] = None
    last_signal = None

    for i in range(len(df)):
        current = df.at[i, columna]
        if current in ["BUY", "SELL"]:
            if current != last_signal:
                df.at[i, "Signal Limpia"] = current
                last_signal = current
            else:
                df.at[i, "Signal Limpia"] = None
        else:
            df.at[i, "Signal Limpia"] = None
    return df


# --- Función Momentum Integral ---
def calcular_momentum_integral(df, window=8):
    df = df.copy()
    df["momentum"] = df["Close"].diff()
    df["integral_momentum"] = df["momentum"].rolling(window=window).sum()
    df["slope_integral"] = df["integral_momentum"].diff()
    std_slope = df["slope_integral"].rolling(window=window).std()

    df["Signal Final"] = np.select(
        [
            (df["slope_integral"] < -std_slope) & (df["momentum"] < 0),
            (df["slope_integral"] > std_slope) & (df["momentum"] > 0)
        ],
        ["SELL", "BUY"],
        default=None
    )
    return df


# --- Función Flow Confirm ---
def confirmar_signal_con_flowtrend(df, flow_mean, flow_threshold=0.15, alignment_weight=1.0):
    df = df.copy()
    df["Flow Confirm"] = "⏸️ Sin señal"
    for i in range(len(df)):
        sig = df.at[i, "Signal Limpia"]
        if sig == "BUY":
            if flow_mean > flow_threshold * alignment_weight:
                df.at[i, "Flow Confirm"] = "BUY ✅ Confirmado"
            else:
                df.at[i, "Flow Confirm"] = "BUY ⚠️ Dudoso"
        elif sig == "SELL":
            if flow_mean < -flow_threshold * alignment_weight:
                df.at[i, "Flow Confirm"] = "SELL ✅ Confirmado"
            else:
                df.at[i, "Flow Confirm"] = "SELL ⚠️ Dudoso"
    return df


# --- Cargar datos ---
data_full = pd.read_pickle("data_full_5coins_with_flow.pkl")
symbol = "BTCUSDT"
df = data_full[symbol]["df"]
flow_mean = data_full[symbol]["flow_mean"]

# --- Aplicar pipeline ---
df = calcular_momentum_integral(df, window=8)
df = limpiar_señales_persistentes(df, columna="Signal Final")

df_flow = confirmar_signal_con_flowtrend(df, flow_mean, flow_threshold=0.15, alignment_weight=1.0)
df_flow = limpiar_señales_persistentes(df_flow, columna="Flow Confirm")

# ==========================================================
# 📤 Exportar a Excel para revisión (sin timezone)
# ==========================================================
excel_file = f"signals_{symbol}_base_vs_flow.xlsx"

# Copias limpias sin timezone
df_export_base = df.copy()
df_export_flow = df_flow.copy()

df_export_base["Open time"] = df_export_base["Open time"].dt.tz_localize(None)
df_export_flow["Open time"] = df_export_flow["Open time"].dt.tz_localize(None)

with pd.ExcelWriter(excel_file, engine="openpyxl") as writer:
    df_export_base[["Open time", "Close", "Signal Final", "Signal Limpia"]].to_excel(
        writer, index=False, sheet_name="Momentum_Base"
    )
    df_export_flow[["Open time", "Close", "Flow Confirm", "Signal Limpia"]].to_excel(
        writer, index=False, sheet_name="Flow_Confirmed"
    )

print(f"✅ Archivo generado correctamente: {excel_file}")
print("→ Revisa las hojas 'Momentum_Base' y 'Flow_Confirmed' en Excel")


✅ Archivo generado correctamente: signals_BTCUSDT_base_vs_flow.xlsx
→ Revisa las hojas 'Momentum_Base' y 'Flow_Confirmed' en Excel


In [45]:


df_base = pd.read_excel("signals_BTCUSDT_base_vs_flow.xlsx", sheet_name="Momentum_Base")
df_flow = pd.read_excel("signals_BTCUSDT_base_vs_flow.xlsx", sheet_name="Flow_Confirmed")

print(df_base.head())
print(df_flow.head())


            Open time     Close Signal Final Signal Limpia
0 2025-05-02 02:00:00  96928.97          NaN           NaN
1 2025-05-02 06:00:00  97330.19          NaN           NaN
2 2025-05-02 10:00:00  96924.23          NaN           NaN
3 2025-05-02 14:00:00  96887.14          NaN           NaN
4 2025-05-02 18:00:00  96337.50          NaN           NaN
            Open time     Close  Flow Confirm  Signal Limpia
0 2025-05-02 02:00:00  96928.97  ⏸️ Sin señal            NaN
1 2025-05-02 06:00:00  97330.19  ⏸️ Sin señal            NaN
2 2025-05-02 10:00:00  96924.23  ⏸️ Sin señal            NaN
3 2025-05-02 14:00:00  96887.14  ⏸️ Sin señal            NaN
4 2025-05-02 18:00:00  96337.50  ⏸️ Sin señal            NaN


In [4]:
# ==========================================================
# 🧩 BLOQUE 0 — Descarga actualizada de velas + flow imbalance
# ==========================================================


# --- 1️⃣ Configuración general ---
symbols = ["BTCUSDT", "ETHUSDT", "ADAUSDT", "XRPUSDT", "BNBUSDT"]
limit = 1000         # número máximo de velas (~6 meses de 4h)
depth_limit = 100    # profundidad del order book
snapshots_n = 10     # número de snapshots para calcular tendencia de flujo
delay_s = 0.5        # pausa entre snapshots para no saturar API
BASE_URL = "https://api.binance.com"

# --- 2️⃣ Función: descarga de velas 4H ---
def get_binance_4h_data(symbol: str, limit: int = 1000) -> pd.DataFrame:
    url = f"{BASE_URL}/api/v3/klines"
    params = {"symbol": symbol, "interval": "4h", "limit": limit}
    r = requests.get(url, params=params)
    r.raise_for_status()
    data = r.json()

    cols = ["Open time","Open","High","Low","Close","Volume",
            "Close time","Quote asset volume","Number of trades",
            "Taker buy base asset volume","Taker buy quote asset volume","Ignore"]
    df = pd.DataFrame(data, columns=cols)

    # Conversión de tipos
    for c in ["Open","High","Low","Close","Volume"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # Conversión de timestamps a CR
    df["Open time"]  = pd.to_datetime(df["Open time"],  unit="ms", utc=True).dt.tz_convert("America/Costa_Rica")
    df["Close time"] = pd.to_datetime(df["Close time"], unit="ms", utc=True).dt.tz_convert("America/Costa_Rica")
    df = df.sort_values("Open time").reset_index(drop=True)
    return df

# --- 3️⃣ Función: cálculo de imbalance del order book ---
def get_orderbook_imbalance(symbol: str, depth: int = 100) -> float:
    url = f"{BASE_URL}/api/v3/depth"
    params = {"symbol": symbol, "limit": depth}
    try:
        r = requests.get(url, params=params, timeout=5)
        r.raise_for_status()
        data = r.json()
        bids = sum(float(b[1]) for b in data["bids"])
        asks = sum(float(a[1]) for a in data["asks"])
        if bids + asks == 0:
            return 0.0
        return round((bids - asks) / (bids + asks), 4)
    except Exception as e:
        print(f"⚠️ {symbol}: error en orderbook → {e}")
        return 0.0

# --- 4️⃣ Función: medir flujo medio y tendencia global ---
def snapshot_orderbook(symbol: str, n=10, delay=0.5):
    vals = []
    for _ in range(n):
        vals.append(get_orderbook_imbalance(symbol, depth_limit))
        time.sleep(delay)
    return [{"timestamp": time.time(), "value": v} for v in vals]

def compute_flow_trend(snapshots):
    vals = [s["value"] for s in snapshots if s.get("value") is not None]
    if len(vals) < 2:
        return 0.0
    return np.clip(vals[-1] - vals[0], -1, 1)

# --- 5️⃣ Función: procesar símbolo completo ---
def fetch_symbol_data(symbol):
    print(f"📥 {symbol} → descargando velas y orderbook...")
    df = get_binance_4h_data(symbol, limit)

    # Imbalance por vela (una lectura por fila, rápida)
    flow_vals = []
    for _ in range(len(df)):
        flow_vals.append(get_orderbook_imbalance(symbol, depth_limit))
        time.sleep(0.1)
    df["Flow_Imbalance"] = flow_vals

    # Snapshots globales
    snapshots = snapshot_orderbook(symbol, n=snapshots_n, delay=delay_s)
    flow_mean = np.mean([s["value"] for s in snapshots])
    flow_trend = compute_flow_trend(snapshots)

    print(f"✅ {symbol}: {len(df)} velas | flow_mean={flow_mean:.3f} | flow_trend={flow_trend:.3f}")
    return symbol, {"df": df, "flow_snapshots": snapshots,
                    "flow_mean": flow_mean, "flow_trend_score": flow_trend}

# --- 6️⃣ Ejecución paralela (todos los símbolos) ---
print("⚡ Descargando data y flow imbalance actualizados...\n")
start = time.time()
data_full = {}

with ThreadPoolExecutor(max_workers=len(symbols)) as ex:
    futures = [ex.submit(fetch_symbol_data, s) for s in symbols]
    for f in as_completed(futures):
        sym, info = f.result()
        data_full[sym] = info

print(f"\n💾 Dataset actualizado para {len(data_full)} símbolos — listo para análisis")
print(f"⏱️ Tiempo total: {time.time() - start:.1f} s")


⚡ Descargando data y flow imbalance actualizados...

📥 BTCUSDT → descargando velas y orderbook...
📥 ETHUSDT → descargando velas y orderbook...
📥 ADAUSDT → descargando velas y orderbook...
📥 XRPUSDT → descargando velas y orderbook...
📥 BNBUSDT → descargando velas y orderbook...
✅ XRPUSDT: 1000 velas | flow_mean=0.196 | flow_trend=0.074
✅ BTCUSDT: 1000 velas | flow_mean=-0.364 | flow_trend=-0.214
✅ BNBUSDT: 1000 velas | flow_mean=0.020 | flow_trend=0.066
✅ ETHUSDT: 1000 velas | flow_mean=0.037 | flow_trend=-0.130
✅ ADAUSDT: 1000 velas | flow_mean=0.131 | flow_trend=0.054

💾 Dataset actualizado para 5 símbolos — listo para análisis
⏱️ Tiempo total: 493.6 s


In [7]:
# ==========================================================
# 🧩 COMPARACIÓN FINAL: Momentum Integral vs Momentum+Flow
# ==========================================================
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import plotly.graph_objects as go

# --- 1️⃣ Configuración general ---
# ⚠️ data_full ya fue generado en el Bloque 0 (con datos actualizados)
#    No lo cargamos desde archivo.
# data_full = pd.read_pickle("data_full_5coins_with_flow.pkl")  ← ❌ eliminar o comentar

pesos = {"BTCUSDT":0.35, "ETHUSDT":0.25, "ADAUSDT":0.10, "XRPUSDT":0.20, "BNBUSDT":0.10}
window = 8   
flow_threshold = 0.15
alignment_weight = 1.0
capital_inicial = 710


# --- 2️⃣ Funciones base ---
def calcular_momentum_integral(df, window=8):
    df = df.copy()
    df["momentum"] = df["Close"].diff()
    df["integral_momentum"] = df["momentum"].rolling(window=window).sum()
    df["slope_integral"] = df["integral_momentum"].diff()
    std_slope = df["slope_integral"].rolling(window=window).std()
    df["Signal Final"] = np.select(
        [
            (df["slope_integral"] < -std_slope) & (df["momentum"] < 0),
            (df["slope_integral"] > std_slope) & (df["momentum"] > 0)
        ],
        ["SELL", "BUY"],
        default=None
    )
    return df

def limpiar_señales_persistentes(df, columna="Signal Final"):
    df = df.copy().reset_index(drop=True)   # ✅ asegura índices consecutivos
    df["Signal Limpia"] = None
    last_signal = None
    for i in range(len(df)):
        current = df.at[i, columna]
        if current in ["BUY", "SELL"]:
            if current != last_signal:
                df.at[i, "Signal Limpia"] = current
                last_signal = current
    return df


def confirmar_signal_con_flowtrend(df, flow_mean, flow_threshold=0.15, alignment_weight=1.0):
    df = df.copy().reset_index(drop=True)   # ✅ asegura índices consecutivos
    df["Flow Confirm"] = "⏸️ Sin señal"
    for i in range(len(df)):
        sig = df.at[i, "Signal Limpia"]
        if sig == "BUY":
            if flow_mean > flow_threshold * alignment_weight:
                df.at[i, "Flow Confirm"] = "BUY ✅ Confirmado"
        elif sig == "SELL":
            if flow_mean < -flow_threshold * alignment_weight:
                df.at[i, "Flow Confirm"] = "SELL ✅ Confirmado"
    return df


def backtest_symbol(df, col_buy, col_sell, rr_tp=1.5, rr_sl=1.0, capital_inicial=710):
    capital = capital_inicial
    en_trade = False
    precio_entrada = 0.0
    for _, row in df.iterrows():
        price = row["Close"]
        if not en_trade and row[col_buy]:
            precio_entrada = price
            en_trade = True
        elif en_trade and row[col_sell]:
            cambio_pct = (price - precio_entrada) / precio_entrada
            if cambio_pct >= rr_tp/100:
                capital *= 1 + rr_tp/100
            elif cambio_pct <= -rr_sl/100:
                capital *= 1 - rr_sl/100
            en_trade = False
    return round(capital, 2)

# --- 3️⃣ Procesamiento por símbolo ---
def procesar_symbol(symbol, info):
    df = info["df"].copy()
    flow_mean = info["flow_mean"]

    # Filtrar últimos 30 días
    fecha_limite = pd.Timestamp.now(tz=df["Open time"].dt.tz) - pd.Timedelta(days=30)
    df = df[df["Open time"] >= fecha_limite].copy()

    # Modelo base (Momentum Integral)
    df_m = calcular_momentum_integral(df, window)
    df_m = limpiar_señales_persistentes(df_m, "Signal Final")
    df_m["Buy"] = df_m["Signal Limpia"] == "BUY"
    df_m["Sell"] = df_m["Signal Limpia"] == "SELL"
    cap_momentum = backtest_symbol(df_m, "Buy", "Sell", rr_tp=2.5, rr_sl=0.5, capital_inicial=capital_inicial)

    # Modelo optimizado con Flow
    df_f = confirmar_signal_con_flowtrend(df_m, flow_mean, flow_threshold, alignment_weight)
    df_f["Buy"] = df_f["Flow Confirm"].str.contains("BUY ✅", na=False)
    df_f["Sell"] = df_f["Flow Confirm"].str.contains("SELL ✅", na=False)
    cap_flow = backtest_symbol(df_f, "Buy", "Sell", rr_tp=2.5, rr_sl=0.5, capital_inicial=capital_inicial)

    return symbol, cap_momentum, cap_flow

# --- 4️⃣ Ejecutar en paralelo ---
print("⚙️ Ejecutando comparación Momentum vs Flow (últimos 30 días)...\n")
start = pd.Timestamp.now()
results = {}

with ThreadPoolExecutor(max_workers=len(pesos)) as ex:
    for sym, cap_m, cap_f in ex.map(lambda s: procesar_symbol(s, data_full[s]), pesos.keys()):
        results[sym] = {"Momentum Integral": cap_m, "Momentum+Flow": cap_f, "Peso": pesos[sym]}

# --- 5️⃣ Resultado consolidado ---
df_res = pd.DataFrame(results).T
for col in ["Momentum Integral", "Momentum+Flow"]:
    df_res[col] = df_res[col].astype(float)

df_res.loc["Portfolio Total"] = {
    "Momentum Integral": np.sum(df_res["Momentum Integral"] * df_res["Peso"]),
    "Momentum+Flow": np.sum(df_res["Momentum+Flow"] * df_res["Peso"]),
    "Peso": 1.0
}

print("\n📊 Resultados Backtest (últimos 30 días):\n")
display(df_res.round(2))
print(f"⏱️ Tiempo total: {(pd.Timestamp.now() - start).total_seconds():.2f} s")

# --- 6️⃣ Visualización (Curva comparativa portafolio) ---
fig = go.Figure()
fig.add_trace(go.Bar(
    x=df_res.index, y=df_res["Momentum Integral"],
    name="Momentum Integral", marker_color="#00cc96"
))
fig.add_trace(go.Bar(
    x=df_res.index, y=df_res["Momentum+Flow"],
    name="Momentum + Flow", marker_color="#ff8800"
))
fig.update_layout(
    barmode="group",
    title="💰 Comparación de Capital Final — Últimos 30 días",
    template="plotly_dark",
    xaxis_title="Símbolo",
    yaxis_title="Capital Final (USD)",
    height=600
)
fig.show()


⚙️ Ejecutando comparación Momentum vs Flow (últimos 30 días)...


📊 Resultados Backtest (últimos 30 días):



,Momentum Integral,Momentum+Flow,Peso
BTCUSDT,749.41,710.0,0.35
ETHUSDT,753.18,710.0,0.25
ADAUSDT,745.67,710.0,0.10
XRPUSDT,738.23,710.0,0.20
BNBUSDT,819.27,710.0,0.10
Portfolio Total,754.73,710.0,1.00


⏱️ Tiempo total: 0.33 s


In [9]:
def confirmar_signal_con_flowtrend(df, flow_threshold=0.15, alignment_weight=1.0, window_flow=12):
    """
    Confirma señales BUY/SELL basándose en la media móvil del flujo (Flow_Imbalance)
    de las últimas N velas, en lugar del flow_mean global.
    """
    df = df.copy().reset_index(drop=True)

    # Calcular media móvil del flow imbalance
    if "Flow_Imbalance" not in df.columns:
        raise ValueError("⚠️ No se encontró la columna 'Flow_Imbalance' en el DataFrame.")
    
    df["Flow_Mean_Roll"] = df["Flow_Imbalance"].rolling(window=window_flow, min_periods=1).mean()

    # Inicializar columna de confirmación
    df["Flow Confirm"] = "⏸️ Sin señal"

    # Confirmación dinámica por vela
    for i in range(len(df)):
        sig = df.at[i, "Signal Limpia"]
        flow_val = df.at[i, "Flow_Mean_Roll"]

        if sig == "BUY":
            if flow_val > flow_threshold * alignment_weight:
                df.at[i, "Flow Confirm"] = "BUY ✅ Confirmado"
        elif sig == "SELL":
            if flow_val < -flow_threshold * alignment_weight:
                df.at[i, "Flow Confirm"] = "SELL ✅ Confirmado"

    return df


In [10]:
# ==========================================================
# 🧩 COMPARACIÓN FINAL: Momentum Integral vs Momentum+Flow (Flow Dinámico)
# ==========================================================
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import plotly.graph_objects as go

# --- 1️⃣ Configuración general ---
# ⚠️ data_full ya fue generado en el Bloque 0 (actualizado hasta hoy)
pesos = {"BTCUSDT":0.35, "ETHUSDT":0.25, "ADAUSDT":0.10, "XRPUSDT":0.20, "BNBUSDT":0.10}
window = 8
flow_threshold = 0.05        # menos estricto (ajustable)
alignment_weight = 0.8       # peso del flujo (ajustable)
capital_inicial = 710
window_flow = 12             # número de velas para promedio móvil del flow

# --- 2️⃣ Funciones base ---
def calcular_momentum_integral(df, window=8):
    df = df.copy()
    df["momentum"] = df["Close"].diff()
    df["integral_momentum"] = df["momentum"].rolling(window=window).sum()
    df["slope_integral"] = df["integral_momentum"].diff()
    std_slope = df["slope_integral"].rolling(window=window).std()
    df["Signal Final"] = np.select(
        [
            (df["slope_integral"] < -std_slope) & (df["momentum"] < 0),
            (df["slope_integral"] > std_slope) & (df["momentum"] > 0)
        ],
        ["SELL", "BUY"],
        default=None
    )
    return df


def limpiar_señales_persistentes(df, columna="Signal Final"):
    df = df.copy().reset_index(drop=True)
    df["Signal Limpia"] = None
    last_signal = None
    for i in range(len(df)):
        current = df.at[i, columna]
        if current in ["BUY", "SELL"]:
            if current != last_signal:
                df.at[i, "Signal Limpia"] = current
                last_signal = current
    return df


def confirmar_signal_con_flowtrend(df, flow_threshold=0.05, alignment_weight=0.8, window_flow=12):
    """
    Confirma señales BUY/SELL basándose en la media móvil del flujo (Flow_Imbalance)
    de las últimas N velas, en lugar del flow_mean global.
    """
    df = df.copy().reset_index(drop=True)
    if "Flow_Imbalance" not in df.columns:
        raise ValueError("⚠️ No se encontró la columna 'Flow_Imbalance' en el DataFrame.")
    
    # Promedio móvil del flujo
    df["Flow_Mean_Roll"] = df["Flow_Imbalance"].rolling(window=window_flow, min_periods=1).mean()

    df["Flow Confirm"] = "⏸️ Sin señal"
    for i in range(len(df)):
        sig = df.at[i, "Signal Limpia"]
        flow_val = df.at[i, "Flow_Mean_Roll"]

        if sig == "BUY":
            if flow_val > flow_threshold * alignment_weight:
                df.at[i, "Flow Confirm"] = "BUY ✅ Confirmado"
        elif sig == "SELL":
            if flow_val < -flow_threshold * alignment_weight:
                df.at[i, "Flow Confirm"] = "SELL ✅ Confirmado"
    return df


def backtest_symbol(df, col_buy, col_sell, rr_tp=2.5, rr_sl=0.5, capital_inicial=710):
    capital = capital_inicial
    en_trade = False
    precio_entrada = 0.0
    for _, row in df.iterrows():
        price = row["Close"]
        if not en_trade and row[col_buy]:
            precio_entrada = price
            en_trade = True
        elif en_trade and row[col_sell]:
            cambio_pct = (price - precio_entrada) / precio_entrada
            if cambio_pct >= rr_tp/100:
                capital *= 1 + rr_tp/100
            elif cambio_pct <= -rr_sl/100:
                capital *= 1 - rr_sl/100
            en_trade = False
    return round(capital, 2)

# --- 3️⃣ Procesamiento por símbolo ---
def procesar_symbol(symbol, info):
    df = info["df"].copy()
    fecha_limite = pd.Timestamp.now(tz=df["Open time"].dt.tz) - pd.Timedelta(days=30)
    df = df[df["Open time"] >= fecha_limite].copy().reset_index(drop=True)

    # Modelo base (Momentum Integral)
    df_m = calcular_momentum_integral(df, window)
    df_m = limpiar_señales_persistentes(df_m, "Signal Final")
    df_m["Buy"] = df_m["Signal Limpia"] == "BUY"
    df_m["Sell"] = df_m["Signal Limpia"] == "SELL"
    cap_momentum = backtest_symbol(df_m, "Buy", "Sell", rr_tp=2.5, rr_sl=0.5, capital_inicial=capital_inicial)

    # Modelo optimizado con Flow dinámico
    df_f = confirmar_signal_con_flowtrend(df_m, flow_threshold, alignment_weight, window_flow)
    df_f["Buy"] = df_f["Flow Confirm"].str.contains("BUY ✅", na=False)
    df_f["Sell"] = df_f["Flow Confirm"].str.contains("SELL ✅", na=False)
    cap_flow = backtest_symbol(df_f, "Buy", "Sell", rr_tp=2.5, rr_sl=0.5, capital_inicial=capital_inicial)

    # Contadores de señales
    total_buy = df_f["Buy"].sum()
    total_sell = df_f["Sell"].sum()

    return symbol, cap_momentum, cap_flow, total_buy, total_sell


# --- 4️⃣ Ejecución paralela ---
print("⚙️ Ejecutando comparación Momentum vs Flow Dinámico (últimos 30 días)...\n")
start = pd.Timestamp.now()
results = {}

with ThreadPoolExecutor(max_workers=len(pesos)) as ex:
    for sym, cap_m, cap_f, total_buy, total_sell in ex.map(lambda s: procesar_symbol(s, data_full[s]), pesos.keys()):
        results[sym] = {
            "Momentum Integral": cap_m,
            "Momentum+Flow": cap_f,
            "BUY Signals": total_buy,
            "SELL Signals": total_sell,
            "Peso": pesos[sym]
        }

# --- 5️⃣ Resultado consolidado ---
df_res = pd.DataFrame(results).T
for col in ["Momentum Integral", "Momentum+Flow"]:
    df_res[col] = df_res[col].astype(float)

df_res.loc["Portfolio Total"] = {
    "Momentum Integral": np.sum(df_res["Momentum Integral"] * df_res["Peso"]),
    "Momentum+Flow": np.sum(df_res["Momentum+Flow"] * df_res["Peso"]),
    "BUY Signals": df_res["BUY Signals"].sum(),
    "SELL Signals": df_res["SELL Signals"].sum(),
    "Peso": 1.0
}

print("\n📊 Resultados Backtest (últimos 30 días con Flow dinámico):\n")
display(df_res.round(2))
print(f"⏱️ Tiempo total: {(pd.Timestamp.now() - start).total_seconds():.2f} s")

# --- 6️⃣ Visualización ---
fig = go.Figure()
fig.add_trace(go.Bar(
    x=df_res.index, y=df_res["Momentum Integral"],
    name="Momentum Integral", marker_color="#00cc96"
))
fig.add_trace(go.Bar(
    x=df_res.index, y=df_res["Momentum+Flow"],
    name="Momentum + Flow (Dinámico)", marker_color="#ff8800"
))
fig.update_layout(
    barmode="group",
    title="💰 Comparación de Capital Final — Flow Dinámico (últimos 30 días)",
    template="plotly_dark",
    xaxis_title="Símbolo",
    yaxis_title="Capital Final (USD)",
    height=600
)
fig.show()


⚙️ Ejecutando comparación Momentum vs Flow Dinámico (últimos 30 días)...


📊 Resultados Backtest (últimos 30 días con Flow dinámico):



,Momentum Integral,Momentum+Flow,BUY Signals,SELL Signals,Peso
BTCUSDT,749.41,710.00,0.0,10.0,0.35
ETHUSDT,753.18,706.45,6.0,4.0,0.25
ADAUSDT,745.67,710.00,12.0,0.0,0.10
XRPUSDT,738.23,710.00,13.0,0.0,0.20
BNBUSDT,819.27,706.45,1.0,5.0,0.10
Portfolio Total,754.73,708.76,32.0,19.0,1.00


⏱️ Tiempo total: 0.37 s


In [16]:
# ==========================================================
# 🧠 BINANCE EXECUTOR — BUY + OCO SELL
# ==========================================================
import os
import math
import logging
from binance.client import Client
from binance.exceptions import BinanceAPIException, BinanceOrderException

# --- Configuración de logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s | %(levelname)s | %(message)s')

# --- Conexión a Binance ---
API_KEY = os.getenv("BINANCE_API_KEY")
API_SECRET = os.getenv("BINANCE_API_SECRET")
client = Client(API_KEY, API_SECRET, tld="us")  # usa tld="com" si estás en binance global

# --- Configuración de pesos ---
WEIGHTS = {
    "BTCUSDT": 0.40,
    "ETHUSDT": 0.30,
    "ADAUSDT": 0.10,
    "XRPUSDT": 0.20
}

CAPITAL_TOTAL_USDT = 710  # capital disponible total


class BinanceTrader:
    def __init__(self, client, capital_total=CAPITAL_TOTAL_USDT, weights=WEIGHTS):
        self.client = client
        self.capital_total = capital_total
        self.weights = weights

    def get_balance(self, asset="USDT"):
        """Obtiene el balance disponible"""
        try:
            bal = self.client.get_asset_balance(asset=asset)
            return float(bal['free'])
        except Exception as e:
            logging.error(f"Error getting balance: {e}")
            return 0.0

    def calculate_quantity(self, symbol, entry_price):
        """Calcula la cantidad a comprar según el peso asignado"""
        weight = self.weights.get(symbol, 0)
        usdt_amount = self.capital_total * weight
        qty = usdt_amount / entry_price
        return round(qty, 6)  # ajusta precisión según par

    def execute_buy(self, symbol, entry, tp, sl):
        """Ejecuta compra y coloca OCO sell"""
        try:
            balance = self.get_balance("USDT")
            qty = self.calculate_quantity(symbol, entry)

            required = qty * entry
            if balance < required:
                logging.warning(f"⚠️ Insufficient USDT balance: need {required:.2f}, have {balance:.2f}")
                return

            # --- Compra a mercado ---
            order = self.client.order_market_buy(symbol=symbol, quantity=qty)
            logging.info(f"✅ BUY executed for {symbol} | qty={qty} | price≈{entry}")

            # --- Orden OCO (Take Profit + Stop Loss) ---
            tp_price = tp
            sl_trigger = round(sl * 1.0003, 2)  # levemente arriba del SL
            sl_limit = sl

            oco = self.client.create_oco_order(
                symbol=symbol,
                side="SELL",
                quantity=qty,
                price=tp_price,
                stopPrice=sl_trigger,
                stopLimitPrice=sl_limit,
                stopLimitTimeInForce="GTC"
            )
            logging.info(f"📉 OCO SELL set: TP={tp_price} | SL={sl_limit}")

        except BinanceAPIException as e:
            logging.error(f"Binance API error: {e.message}")
        except BinanceOrderException as e:
            logging.error(f"Order error: {e.message}")
        except Exception as e:
            logging.error(f"Unexpected error: {e}")




In [ ]:
# ==========================================================
# 🚀 Ejemplo de uso (simulado)
# ==========================================================
if __name__ == "__main__":
    trader = BinanceTrader(client)

    # Ejemplo real con datos de tu señal
    trader.execute_buy(
        symbol="BTCUSDT",
        entry=113_426.55,
        tp=122_287.785,
        sl=107_519.06
    )


In [18]:
# ==========================================================
# 📈 BTCUSDT — Dirección (Momentum) y Velocidad (Aceleración)
# ==========================================================
import numpy as np
import plotly.graph_objects as go

# --- 1️⃣ Dataset real desde data_full ---
df_btc = data_full["BTCUSDT"]["df"].copy()
df_btc = df_btc.sort_values("Open time").reset_index(drop=True)

# --- 2️⃣ Calcular derivadas ---
df_btc["Momentum"] = np.gradient(df_btc["Close"])
df_btc["Acceleration"] = np.gradient(df_btc["Momentum"])

# --- 3️⃣ Determinar zonas alcistas/bajistas según el signo del momentum ---
df_btc["Trend"] = np.where(df_btc["Momentum"] >= 0, "Bullish", "Bearish")

# --- 4️⃣ Detectar cambios de dirección (puntos críticos) ---
critical_points = np.where(np.diff(np.sign(df_btc["Momentum"])))[0]
max_points = [i for i in critical_points if df_btc["Acceleration"].iloc[i] < 0]
min_points = [i for i in critical_points if df_btc["Acceleration"].iloc[i] > 0]

# --- 5️⃣ Crear gráfico con fondo coloreado según dirección ---
fig = go.Figure()

# Fondo de color según momentum
for i in range(1, len(df_btc)):
    color = "rgba(0,255,0,0.08)" if df_btc["Trend"].iloc[i] == "Bullish" else "rgba(255,0,0,0.08)"
    fig.add_vrect(
        x0=df_btc["Open time"].iloc[i-1], x1=df_btc["Open time"].iloc[i],
        fillcolor=color, line_width=0, opacity=0.4, layer="below"
    )

# Precio BTCUSDT (eje izquierdo)
fig.add_trace(go.Scatter(
    x=df_btc["Open time"], y=df_btc["Close"],
    name="Precio BTCUSDT",
    line=dict(color="white", width=3),
    yaxis="y1"
))

# Aceleración (eje derecho)
fig.add_trace(go.Scatter(
    x=df_btc["Open time"], y=df_btc["Acceleration"],
    name="Aceleración (2ª derivada)",
    line=dict(color="deepskyblue", width=2, dash="dot"),
    yaxis="y2"
))

# Flechas o marcadores para los puntos críticos
fig.add_trace(go.Scatter(
    x=df_btc.loc[max_points, "Open time"],
    y=df_btc.loc[max_points, "Close"],
    mode="markers",
    name="Máximos locales",
    marker=dict(color="red", size=10, symbol="triangle-down"),
    yaxis="y1"
))
fig.add_trace(go.Scatter(
    x=df_btc.loc[min_points, "Open time"],
    y=df_btc.loc[min_points, "Close"],
    mode="markers",
    name="Mínimos locales",
    marker=dict(color="lime", size=10, symbol="triangle-up"),
    yaxis="y1"
))

# --- 6️⃣ Layout final ---
fig.update_layout(
    template="plotly_dark",
    title="BTCUSDT — Dirección (Momentum) y Velocidad (Aceleración)",
    xaxis=dict(title="Tiempo (intervalos 4H)"),
    yaxis=dict(title="Precio BTCUSDT", side="left", showgrid=False),
    yaxis2=dict(
        title="Aceleración (2ª derivada)",
        overlaying="y",
        side="right",
        showgrid=False
    ),
    legend=dict(
        x=1.02, y=1,
        bgcolor="rgba(0,0,0,0)",
        bordercolor="rgba(0,0,0,0)"
    ),
    margin=dict(r=200, t=60, b=60, l=80),
    height=650
)

fig.show()
